<div style='color: #216969;
           background-color: #EAF6F6;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: solid;
           border-color: dark green;
           font-family: "Cambria";'>
Selection Best features  ....
<a class="anchor" id="1"></a> 

<img src="https://miro.medium.com/max/594/1*nyC9bOwnK16Cl8jFYTyEFQ.png" width=400>

- Data loading
- Data Imputation
- Lasso deployement 
= Model Evaluation


https://youtu.be/T4nZDuakYlU?list=PLO_fdPEVlfKqMDNmCFzQISI2H_nJcEDJq

https://youtu.be/xlHk4okO8Ls

https://towardsdatascience.com/feature-selection-in-python-recursive-feature-elimination-19f1c39b8d15

https://github.com/Yimeng-Zhang/feature-engineering-and-feature-selection/blob/master/4.5_Demo_Feature_Selection_Hybrid_method.ipynb



Lasso Regression (L1 Regularization)

What it is: Lasso (Least Absolute Shrinkage and Selection Operator) regression is a linear regression technique that adds a penalty term to the ordinary least squares (OLS) cost function. This penalty is proportional to the absolute value of the coefficients.   

Why it's used:

Regularization: The penalty term shrinks the coefficients of less important features towards zero. This helps prevent overfitting, especially when you have many features or when some features are highly correlated.
Feature Selection: A key feature of Lasso is that it can force some coefficients to become exactly zero. This effectively performs feature selection, automatically identifying and discarding irrelevant features. This is different from Ridge regression (L2 regularization), which shrinks coefficients but rarely sets them to zero.
The Cost Function: The cost function Lasso minimizes is:

RSS + alpha * sum(|beta_j|)
 Where:

RSS is the residual sum of squares (the standard OLS term).
alpha is the regularization parameter (a hyperparameter you need to tune). A higher alpha means stronger regularization (more shrinkage and more coefficients set to zero).
beta_j are the regression coefficients.
sum(|beta_j|) is the sum of the absolute values of the coefficients (the L1 penalty).
2. Cross-Validation (CV)

What it is: Cross-validation is a technique for evaluating how well a model will generalize to unseen data. It involves splitting the data into multiple "folds," training the model on some folds (the training set), and evaluating it on the remaining fold (the validation set). This is repeated multiple times, with each fold serving as the validation set once.
Why it's used: It provides a more reliable estimate of model performance than a single train/test split, especially when you have limited data. It helps you choose the best hyperparameters (like alpha in Lasso) without "peeking" at the final test set.
Common Types:
k-fold Cross-Validation: The data is divided into k folds.
Leave-One-Out Cross-Validation (LOOCV): Each data point is used as the validation set once (extreme case of k-fold where k = number of samples).
Stratified k-fold: Ensures that each fold has roughly the same proportion of samples from each class (important for classification problems with imbalanced classes).
3. LassoCV in scikit-learn

sklearn.linear_model.LassoCV: This class in scikit-learn combines Lasso regression with cross-validation to automatically find the best value of the regularization parameter (alpha).
cv parameter:: Determines the cross-validation splitting strategy. If None, to use the default 5-fold cross-validation.
alphas parameter:: List of alphas where to compute the models. If None alphas are set automatically.

In [1]:
#تحميل البيانات
import pandas as pd
df= pd.read_csv("../data/HousePrice3.csv", index_col=0)

In [2]:
df.sample(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
967,968,20,RL,NaN,7390,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,135000
1059,1060,50,RL,NaN,11275,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,220000
1091,1092,160,FV,24.0,2160,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2008,WD,Normal,160000


![](../imgs/featureEngin.png)

## Conduct Recursive Feature Elimination

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 0 to 1459
Data columns (total 78 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallCond  

In [4]:
df.select_dtypes("number").head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [5]:
df.isna().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
MasVnrType        872
                 ... 
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
Length: 78, dtype: int64

In [6]:
df.select_dtypes("number").head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [7]:
# 1. Separate Numerical and Categorical Columns
from sklearn.impute import SimpleImputer
def preprocess_dataframe(df):

    numerical_cols = df.select_dtypes(include=['number']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns  # Or exclude number

    # 2. Impute Missing Values (Numerical)
    if len(numerical_cols) > 0 :
        imputer_num = SimpleImputer(strategy='mean')  # Or 'median'
        df[numerical_cols] = imputer_num.fit_transform(df[numerical_cols])

    # 3. Handle Categorical Columns (One-Hot Encoding)
    if len(categorical_cols) > 0 :
        df = pd.get_dummies(df, columns=categorical_cols, dummy_na=True)  # dummy_na handles NaN in categorical

    return df

df = pd.DataFrame(preprocess_dataframe(df), index = df.index)

In [8]:
df.isna().sum().sort_values(ascending=False)

SaleCondition_nan    0
Id                   0
MSSubClass           0
LotFrontage          0
LotArea              0
                    ..
2ndFlrSF             0
TotalBsmtSF          0
BsmtUnfSF            0
BsmtFinSF2           0
BsmtFinSF1           0
Length: 329, dtype: int64

![](../imgs/dataSplit.jpg)

In [9]:

X=  df.select_dtypes("number").iloc[:,:-1]
y= df.SalePrice

In [10]:
X.shape , y.shape

((1460, 34), (1460,))

In [11]:
# تقسيم البيانات
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y  ,test_size=.33, random_state=42)

![](../imgs/model.png)

### freature Selection using RFECV

In [13]:
# Choose an estimator (model)
from sklearn.linear_model import LinearRegression  #
from sklearn.feature_selection import RFECV
estimator = LinearRegression()  # Or RandomForestRegressor, etc.

# Create RFECV
rfecv = RFECV(
    estimator=estimator,
    step=1,  # Number of features to remove at each iteration
    cv=5,  # Number of cross-validation folds
    scoring='neg_mean_squared_error',  # Or 'accuracy', 'r2', etc.
)

# Fit RFECV
rfecv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), scoring='neg_mean_squared_error')

In [15]:
from sklearn.metrics import mean_squared_error

print("Optimal number of features : %d" % rfecv.n_features_)
print("Selected features: %s" % X_train.columns[rfecv.support_])

# Transform data to selected features
X_train_selected = rfecv.transform(X_train)
X_test_selected = rfecv.transform(X_test)

# Train a model with selected features
model = estimator  # Or create a new instance
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Optimal number of features : 12
Selected features: Index(['MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces'],
      dtype='object')
Mean Squared Error: 1852543329.5656679
